In [ ]:
# Pre Processing
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Paths
data_dir = "./"
RAW_CSV = f"{data_dir}Dataset.csv"
SCALER_PATH = f"{data_dir}scaler_input.pkl"

# 1) Load raw TORCS log data
print("Loading raw data from:", RAW_CSV)
df = pd.read_csv(RAW_CSV)

# 2) Drop countdown / pre-start rows (curLapTime < 0)
df = df[df["CurrentLapTime"] >= 0].reset_index(drop=True)
print("Dropped pre-start rows; remaining rows:", df.shape[0])

# 3) Drop unused / meta columns (keep 'gear' as feature)
drop_cols = [
    "Damage", "CurrentLapTime", "DistanceFromStart",
    "DistanceCovered", "FuelLevel", "LastLapTime",
    "RacePosition"
]
df = df.drop(columns=drop_cols)

# 4) Define feature & label columns, include 'gear' as feature
track_cols = [f"Track_{i}" for i in range(1,20)]
wsv_cols   = [f"WheelSpinVelocity_{i}" for i in range(1,5)]
opp_cols   = [f"Opponent_{i}" for i in range(1,37)]

feature_cols = ["Angle"] + opp_cols + [
    "RPM", "SpeedX", "SpeedY", "SpeedZ", "TrackPosition", "Z", "gear"
] + track_cols + wsv_cols
label_cols   = ["Clutch", "Braking", "Steering", "Acceleration", "Gear"]  # gear removed from label

# 5) Normalize features into [0,1] using MinMaxScaler
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(df[feature_cols])
joblib.dump(scaler, SCALER_PATH)
print("Fitted MinMaxScaler including gear and saved scaler.")

# 6) Prepare labels (gear is now input, so labels exclude it)
y_df = df[label_cols].reset_index(drop=True)

# 7) Convert normalized features back to DataFrame
X_norm_df = pd.DataFrame(X_norm, columns=feature_cols)

# 8) Train/validation split and save
test_size = 0.2
X_train, X_val, y_train, y_val = train_test_split(
    X_norm_df, y_df,
    test_size=test_size,
    random_state=42,
    shuffle=True
)

X_train.to_csv(f"{data_dir}train_X.csv", index=False)
X_val.to_csv(f"{data_dir}val_X.csv", index=False)
y_train.to_csv(f"{data_dir}train_y.csv", index=False)
y_val.to_csv(f"{data_dir}val_y.csv", index=False)

print("Preprocessing complete:")
print(" train_X shape:", X_train.shape)
print(" val_X shape:  ", X_val.shape)
print(" train_y shape:", y_train.shape)
print(" val_y shape:  ", y_val.shape)



In [ ]:
# Training
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Paths
data_dir = "./"
TRAIN_X = f"{data_dir}train_X.csv"
TRAIN_Y = f"{data_dir}train_y.csv"
VAL_X   = f"{data_dir}val_X.csv"
VAL_Y   = f"{data_dir}val_y.csv"
MODEL_PATH = f"{data_dir}trained_model.h5"

# 1) Load preprocessed training and validation data
X_train = pd.read_csv(TRAIN_X).values
y_train = pd.read_csv(TRAIN_Y).values
X_val   = pd.read_csv(VAL_X).values
y_val   = pd.read_csv(VAL_Y).values

# 2) Build a feed-forward neural network with added layers
model = Sequential([
    Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(y_train.shape[1])
])

# 3) Compile model with learning rate 1e-4
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='mse',
    metrics=['mae']
)

# 4) Set up callbacks including LR reduction on plateau
callbacks = [
    ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_loss'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

# 5) Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=callbacks
)

# 6) Save final model
model.save(MODEL_PATH)
print(f"Model training complete. Best model saved to {MODEL_PATH}")
